## ETF Compete:  
## Feature engineering: MACD (Moving Average Convergence/Divergence)

## 有名的MACD 柱狀圖

## Using EMA(Exponential Moving Average)

## $ EMA_{t1} = \alpha * p + (1- \alpha) * EMA_{t0}$

## 26days, $\alpha = 2/(26+1) = 0.074$
## 12days, $\alpha = 2/(26+1) = 0.154$

## 9days, $\alpha = 2/(9+1) = 0.2$

## $p$ = toda's close price


### Date: 18425_L

# 使用變數

$$$$
# EMA26 = 26天移動平均
# EMA12 = 12 天移動平均
# DIF  = MA26 - MA12 
# MACD = EMA(DIF, n), 為 DIF n 日移動平均, 一般取9
# OSC = DIF - MACD (柱狀體) 
# vs_EMA26 = Today close - EMA 26
# vs EMA 12 = Today close - EMA 12





###  參數: 
### df: dataframe 
### n: n days window (坊間書籍用 3 , using 5 here )


###  return df 
### feature columns = [ 'DIF', 'MACD', 'OSC', 'vs_EMA26', 'vs_EMA12' ]
###  

In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict
from time import time

In [2]:
# read in csv, change column names
etf = pd.read_csv('/Users/LarryGuo/Desktop/nano_degree/Capstone_Talk/ETF_compete/twetf_utf8/tetfp.csv')
col_dtypes = OrderedDict(code=str, date=str, name=str, open=float, high=float, low=float, close=float, volume=int)

etf.columns=col_dtypes.keys()

etf.head()

,code,date,name,open,high,low,close,volume
0,50,20130102,元大台灣50,54.00,54.65,53.90,54.40,16487
1,50,20130103,元大台灣50,54.90,55.05,54.65,54.85,29020
2,50,20130104,元大台灣50,54.85,54.85,54.40,54.50,9837
3,50,20130107,元大台灣50,54.55,54.55,53.90,54.25,8910
4,50,20130108,元大台灣50,54.00,54.20,53.65,53.90,12507


In [3]:

etf= etf[:1286] # select 台灣50 
etf.tail()

,code,date,name,open,high,low,close,volume
1281,50,20180327,元大台灣50,83.00,83.40,82.95,83.40,3277
1282,50,20180328,元大台灣50,82.90,82.90,82.20,82.25,4161
1283,50,20180329,元大台灣50,82.25,82.35,81.80,82.10,4099
1284,50,20180330,元大台灣50,82.65,83.05,82.65,82.85,4994
1285,50,20180331,元大台灣50,82.85,83.05,82.75,82.95,878


In [4]:
etf.columns

Index(['code', 'date', 'name', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [5]:
def calc_MACD(df, a_26=0.074, a_12=0.154,a_9=0.2):
    df['EMA26'] = 0
    df['EMA12'] = 0
    df['EMA26'].iloc[0]=df['close'].iloc[0]
    df['EMA12'].iloc[0]=df['close'].iloc[0]
    
    for j in range(1,len(df)):
        df['EMA26'].iloc[j] = a_26* df['close'].iloc[j] + (1-a_26)* df['EMA26'].iloc[j-1]
        df['EMA12'].iloc[j] = a_12* df['close'].iloc[j] + (1-a_12)* df['EMA12'].iloc[j-1]
    df['DIF'] = df['EMA12']  - df['EMA26']
    df['MACD'] = 0
    df['MACD'].iloc[0] = df['DIF'].iloc[0]
    for j in range (1,len(df)):
        df['MACD'].iloc[j] = a_9 * df['DIF'].iloc[j] + (1-a_9)*df['MACD'].iloc[j-1]
    df['OSC'] = df['DIF'] - df ['MACD']
    
    df['vs_EMA26'] = df['close'] - df['EMA26']
    df['vs_EMA12'] = df['close'] - df['EMA12']
    return df
    


 
   

In [6]:
df= calc_MACD(etf)

/Users/LarryGuo/miniconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
df.columns

Index(['code', 'date', 'name', 'open', 'high', 'low', 'close', 'volume',
       'EMA26', 'EMA12', 'DIF', 'MACD', 'OSC', 'vs_EMA26', 'vs_EMA12'],
      dtype='object')

In [8]:
df

,code,date,name,open,high,low,close,volume,EMA26,EMA12,DIF,MACD,OSC,vs_EMA26,vs_EMA12
0,50,20130102,元大台灣50,54.00,54.65,53.90,54.40,16487,54.400000,54.400000,0.000000,0.000000,0.000000,0.000000,0.000000
1,50,20130103,元大台灣50,54.90,55.05,54.65,54.85,29020,54.433300,54.469300,0.036000,0.007200,0.028800,0.416700,0.380700
2,50,20130104,元大台灣50,54.85,54.85,54.40,54.50,9837,54.438236,54.474028,0.035792,0.012918,0.022874,0.061764,0.025972
3,50,20130107,元大台灣50,54.55,54.55,53.90,54.25,8910,54.424306,54.439528,0.015221,0.013379,0.001842,-0.174306,-0.189528
4,50,20130108,元大台灣50,54.00,54.20,53.65,53.90,12507,54.385508,54.356440,-0.029067,0.004890,-0.033957,-0.485508,-0.456440
5,50,20130109,元大台灣50,53.75,54.30,53.75,54.10,7529,54.364380,54.316948,-0.047432,-0.005575,-0.041857,-0.264380,-0.216948
6,50,20130110,元大台灣50,54.30,54.65,54.15,54.50,13953,54.374416,54.345138,-0.029278,-0.010315,-0.018962,0.125584,0.154862
7,50,20130111,元大台灣50,54.70,54.80,54.35,54.45,11837,54.380009,54.361287,-0.018722,-0.011997,-0.006726,0.069991,0.088713
8,50,20130114,元大台灣50,54.00,54.50,53.80,54.50,7282,54.388889,54.382649,-0.006240,-0.010845,0.004606,0.111111,0.117351
9,50,20130115,元大台灣50,54.20,54.45,53.90,54.00,6609,54.360111,54.323721,-0.036390,-0.015954,-0.020436,-0.360111,-0.323721
